In [1]:
#pip install sentence-transformers
import pandas as pd
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-roberta-large')

In [8]:
import spacy
!spacy download en_core_web_md
from math import sqrt, pow, exp
nlp = spacy.load("en_core_web_md")
sentences=['The bottle is empty','There is nothing in the bottle']


In [5]:
def jaccard_similarity(x,y):
    """ returns the jaccard similarity between two lists """
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

jacard_sentences = [sent.lower().split(" ") for sent in sentences]
jaccard_similarity(jacard_sentences[0], jacard_sentences[1])

0.42857142857142855

In [32]:
sentences=['The bottle is empty','There is nothing in the bottle']
embeddings = [nlp(sentence).vector for sentence in sentences]
def squared_sum(x):
    """ return 3 rounded square rooted value """

    return round(sqrt(sum([a*a for a in x])),3)

def euclidean_distance(x,y):
    """ return euclidean distance between two lists """

    return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))


def distance_to_similarity(distance):
    return 1/exp(distance)

distance_to_similarity(euclidean_distance(embeddings[0], embeddings[1]))


def getEmbeddings(x,y,indicator):
    
    sentences=[]
    sentences.append(x)
    sentences.append(y)
    if indicator == 0:
        embeddings = [nlp(sentence).vector for sentence in sentences]
        return cos_similarity(embeddings[0], embeddings[1])
    elif indicator == 1:
        embeddings = model.encode(sentences, convert_to_tensor=True)
        return sentenceTransformer(embeddings)
        
def sentenceTransformer(embed):
    embeddings=embed
    similarity = []
    for i in range(len(sentences)):
        row = []
        for j in range(len(sentences)):
            row.append(util.pytorch_cos_sim(embeddings[i], embeddings[j]).item())
        similarity.append(row)
    return round(similarity[0][1],3)
    
    
    

def cos_similarity(x,y):
    """ return cosine similarity between two lists """
    
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = squared_sum(x)*squared_sum(y)
    return round(numerator/float(denominator),3)



In [48]:
df=pd.read_csv('train.csv')

In [49]:
df=df[:500]

In [50]:
df['CosineSimilarity']=df.apply(lambda x : getEmbeddings(x['question1'],x['question2'],0),axis=1)
df['Sentence Transformer']=df.apply(lambda x : getEmbeddings(x['question1'],x['question2'],1),axis=1)

In [51]:
df['cosine_Result']=df['CosineSimilarity'].apply(lambda x : 1 if x>0.9 else 0)
df['sent_Result']=df['Sentence Transformer'].apply(lambda x : 1 if x>0.85 else 0)
df['sent_Accuracy']=df.apply(lambda x : 1 if x['is_duplicate']==x['sent_Result'] else 0 , axis=1 )

In [52]:
df['sent_Accuracy'].value_counts()

1    371
0    129
Name: sent_Accuracy, dtype: int64

In [37]:
df.to_csv('test.csv',index=False)

In [53]:
df

,id,qid1,qid2,question1,question2,is_duplicate,CosineSimilarity,Sentence Transformer,cosine_Result,sent_Result,sent_Accuracy
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.987,0.729,1,0,1
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.934,0.591,1,0,1
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.929,0.784,1,0,1
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.731,0.112,0,0,1
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.848,0.395,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
495,495,988,989,What is the painting on this image?,What is this painting?,0,0.949,0.935,1,1,0
496,496,990,991,Which are the major highways in California and...,Which are the major highways in California and...,0,0.997,0.778,1,0,1
497,497,992,993,What's beyond our Universe?,"If space is expanding, where does the new spac...",0,0.851,0.688,0,0,1
498,498,994,995,Is growing of hair a physical or a chemical ch...,Can a bald person ever grow their hair back?,0,0.876,0.461,0,0,1
